In [950]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

import warnings
warnings.filterwarnings('ignore')

In [951]:
# data = pd.read_csv('../data/retail_train.csv')
data = pd.read_csv('D:/WORK/BI/GeekBrains/Course_Recommender_systems/Data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [952]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [953]:
def precision_at_k(recommended_list, bought_list, k = 5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    return precision

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар?  
* Зачем он нужен?  
* Используя этот товар мы смещаем качество рекомендаций.
* В какую сторону?   
* Можно ли удалить этот товар?   
* Уберите этот товар (**внимание**: это можно сделать разными способами!) и сравните с качеством на семинаре.

1. Товаром 999999 заменили все непопулярные товары, находящиеся ниже Топ-5000 в рейтинге продаж
2. С введением этого товара качество рекомендацийнезначительно ухудшилось (~ с 0,16 до 0,15), но это некритично.
3. В целом этот товар можно не вводить, но лучше все-таки это сделать. Т.к. время обработки данных увеличивается на порядок.  Для примера, перебор 20 вариантов K-соседей (от 5 до 100) увеличился с нескольких секунд до 7-8 минут
4. При полном удаленни строк с этим товаром из матрицы у меня перестал строится прогноз. Возможно, что-то я делал не так.


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [954]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [955]:
# df_abc_sale = df_groups_stat.groupby('rank_abc').agg({'sum': ['sum'], 'product_group': ['count'], 'share_sale': 'sum'})
df_sales = data.groupby('item_id').agg({
                                        'sales_value': ['sum'], 
                                        'quantity':    ['sum'],
                                        'item_id':     ['count']
                                        })
df_sales.reset_index(inplace = True)
df_sales = df_sales.droplevel(1, axis=1)
df_sales.columns = ['item_id', 'sales_value', 'quantity', 'count_trans']
df_sales['price_avg'] = round(df_sales['sales_value'] / df_sales['quantity'], 2)
df_sales.head(5)

,item_id,sales_value,quantity,count_trans,price_avg
0,25671,20.94,6,3,3.49
1,26081,0.99,1,1,0.99
2,26093,1.59,1,1,1.59
3,26190,1.54,1,1,1.54
4,26355,1.98,2,1,0.99


In [956]:
df_sales

,item_id,sales_value,quantity,count_trans,price_avg
0,25671,20.94,6,3,3.49
1,26081,0.99,1,1,0.99
2,26093,1.59,1,1,1.59
3,26190,1.54,1,1,1.54
4,26355,1.98,2,1,0.99
...,...,...,...,...,...
89046,17991689,2.49,1,1,2.49
89047,17991691,2.49,1,1,2.49
89048,18000012,19.96,4,3,4.99
89049,18024155,3.99,1,1,3.99


In [957]:

# группировка по товарам с итогами: 
#      по обороту, по количеству, по числу транзакций
df_sales = data.groupby('item_id').agg({
                                        'sales_value': ['sum'], 
                                        'quantity':    ['sum'],
                                        'item_id':     ['count']
                                        })
df_sales.reset_index(inplace = True)
df_sales = df_sales.droplevel(1, axis=1)
df_sales.columns = ['item_id', 'sales_value', 'quantity', 'count_trans']

# средневзвешенная цена
df_sales['price_avg'] = 0
df_sales.loc[ df_sales['quantity'] != 0, 'price_avg'] = round(df_sales['sales_value'] / df_sales['quantity'], 8)

# df_sales['price_avg'].fillna(0, inplace = True)
# df_sales.loc[ df_sales['price_avg'] == np.inf ] = df_sales['price_avg'].max()

df_sales['sales_value_1'] = df_sales['sales_value']
df_sales.loc[ df_sales['sales_value'] == 0, 'sales_value_1'] = 1

df_sales['log_sales'] = np.log( df_sales['sales_value_1'] )
df_sales.loc[ df_sales['sales_value'] == 0, 'log_sales'] = 0

# итоговый датасет
df_sales = df_sales[ [ 'item_id', 'sales_value', 'log_sales','quantity', 'count_trans', 'price_avg']]

df_sales.sort_values('sales_value', ascending = False, inplace = True)

print()
print(df_sales.head(5))
print()
print(df_sales[::-5].head(5))


       item_id  sales_value  log_sales   quantity  count_trans  price_avg
56233  6534178    467993.62  13.056210  199684264        18364   0.002344
56193  6533889     42645.75  10.660683   16911359         1332   0.002522
29195  1029743     37981.91  10.544865      15840        13455   2.397848
56228  6534166     31298.96  10.351340   12946508         1125   0.002418
35054  1082185     27291.02  10.214313      28384        27362   0.961493

        item_id  sales_value  log_sales  quantity  count_trans  price_avg
59028   7160577          0.0        0.0         0            2        0.0
25643    997931          0.0        0.0         0            1        0.0
85273  15596526          0.0        0.0         1            1        0.0
83425  13945300          0.0        0.0         1            1        0.0
37602   1105561          0.0        0.0         1            1        0.0


In [958]:
def weighted_random_recommendation(items_weights, N = 5):
    """ Взвешенные Случайные рекоммендации
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    
    items = np.array(items_weights.iloc[:, 0])
    prob  = np.array(items_weights.iloc[:, 1])
    
    # нормализация массива значений весов
    w_min = np.min(prob)
    w_max = np.max(prob)
    
    prob = (prob - w_min) / (w_max - w_min)
    w_sum = np.sum(prob)
    
    prob = prob / w_sum

    recs = np.random.choice( items, size = N, replace = False, p = prob)
    
    return recs.tolist()

Сделайте предсказания

**Вес = Log(sales_value)**

In [959]:
%%time

df = df_sales[['item_id', 'log_sales']]
print(weighted_random_recommendation(df, N = 5))

[932020, 13776075, 1038998, 975957, 1002451]
Wall time: 12.9 ms


**Вес = quantity**

In [960]:
%%time

df = df_sales[['item_id', 'quantity']]
print(weighted_random_recommendation(df, N = 5))

[6534178, 397896, 6533889, 6544236, 6534166]
Wall time: 13 ms


**Вес = count_trans**

In [961]:
%%time

df = df_sales[['item_id', 'count_trans']]
print(weighted_random_recommendation(df, N = 5))

[1031864, 896369, 874255, 1044845, 1108168]
Wall time: 20.9 ms


**Вес = price_avg**

In [962]:
%%time

df = df_sales[['item_id', 'price_avg']]
print(weighted_random_recommendation(df, N = 5))

[7441574, 15716050, 6536271, 905972, 6772836]
Wall time: 6.98 ms


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1.  Какой алгоритм показывает лучшее качество? Почему?

In [963]:
result = pd.read_csv('preds.csv') # загрузка predict с семикнара
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1005456, 6942901, 10149303, 9424406, 904199]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[9803693, 1095915, 949877, 9884766, 6961764]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [964]:
# перевод строк с предсказаниями в список
for a in result.columns[1:]:
    result[a] = result[a].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))


**Еще раз, взвешенные случайные рекомендации, Вес = log(count_trans)**

Добавляю их в result

In [965]:
df = df_sales[['item_id', 'count_trans']]

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(df, N = 5))
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1005456, 6942901, 10149303, 9424406, 904199]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[991546, 838797, 7441180, 838396, 975938]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[9803693, 1095915, 949877, 9884766, 6961764]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[976199, 1024231, 8090509, 972657, 908318]"


#### Оценка качества алгоритмов

In [966]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1005456, 6942901, 10149303, 9424406, 904199]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[991546, 838797, 7441180, 838396, 975938]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[9803693, 1095915, 949877, 9884766, 6961764]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[976199, 1024231, 8090509, 972657, 908318]"


In [967]:
#Вывод рейтинга оценки качества алгоритмов
def show_alg_result(alg_list):

    result_test = []

    for alg in alg_list:
        res = result.apply(lambda x: precision_at_k( x[alg], x['actual'],  5), axis = 1 ).mean()
        result_test.append( [res, alg])

        result_test.sort(key = lambda x: x[0], reverse = True)

    print(f'Место  Качество  Алгоритм')
    print()
    for place, res in enumerate(result_test):
        print(f'   {place + 1}   {round(res[0], 5)}\t {res[1]}')
        
    return result_test

In [968]:
alg_list = ['random_recommendation',
                'weighted_random_recommendation',
                'popular_recommendation',
                'itemitem',
                'cosine',
                'tfidf',
                'own_purchases']

result_test = show_alg_result(alg_list)

Место  Качество  Алгоритм

   1   0.17969	 own_purchases
   2   0.15524	 popular_recommendation
   3   0.13898	 tfidf
   4   0.13692	 itemitem
   5   0.13291	 cosine
   6   0.02165	 weighted_random_recommendation
   7   0.00127	 random_recommendation


#### Выводы

Лучшее качество показали алгоритмы, связанные с популярностью товаров.
* Первое место у алгоритма **own_purchases** - самые популярные товары по каждому пользователю.  
* На втором месте алгоритм **popular_recommendation** - самые популярные товары по всем пользователям.  
* Худшие результаты у алгоритмов, использующих случайный выбор.


### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


In [969]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns = {'quantity': 'n_sold'}, inplace = True)

popularity.shape, popularity.head(10)

((86865, 2),
    item_id  n_sold
 0    25671       6
 1    26081       1
 2    26093       1
 3    26190       1
 4    26355       2
 5    26426       1
 6    26540       3
 7    26601       1
 8    26636       1
 9    26691       3)

In [970]:
top_5000 = popularity.sort_values('n_sold', ascending = False).head(5000).item_id.tolist()

In [971]:
# Заведем фиктивный item_id (если юзер НЕ покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999
data_train.shape

(2278490, 12)

In [972]:
# data_train = data_train.loc[ data_train['item_id'] != 999999 ]
# data_train.shape

In [973]:
# сводная таблица
user_item_matrix = pd.pivot_table(data_train, 
                                      index = 'user_id', columns='item_id', 
                                      values = 'quantity', aggfunc = 'count', 
                                      fill_value = 0
                                 )

user_item_matrix[user_item_matrix > 0] = 1           # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float)    # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [974]:
# Разреженность матрицы (с товаром 999999)
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.33770796861036

In [975]:
# перенумеруем пользователей и товары
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [976]:
# новый датасет с результами
result_new = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_new.columns = ['user_id', 'actual']
result_new['actual'] = result_new['actual'].apply(lambda x: list(x))
result_new.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


In [977]:
%%time

print('Перебор K-соседей от 5 до 100:')
K = 0
for i in range(3, 21, 1):

    K = i * 5
    if (K % 25 != 0):
        print(f'\tK_{K}', end = " ")
    else:
        print(f'\tK_{K}')
    
    model = ItemItemRecommender(K, num_threads = 2)                               # K - кол-во ближайших соседей
    model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress = False)      # На вход item-user matrix

    result_new[f'K_{K}'] = result_new['user_id'].apply(
                                        lambda x: [ id_to_itemid[ rec[0] ] for rec in 
                                            model.recommend( userid = userid_to_id[x], 
                                                            user_items = sparse_user_item,   # на вход user-item matrix
                                                            N = 5, 
                                                            filter_already_liked_items = False, 
                                                            filter_items = None, 
                                                            recalculate_user = True)]) 
   

Перебор K-соседей от 5 до 100:
	K_15 	K_20 	K_25
	K_30 	K_35 	K_40 	K_45 	K_50
	K_55 	K_60 	K_65 	K_70 	K_75
	K_80 	K_85 	K_90 	K_95 	K_100
Wall time: 31.5 s


In [978]:
result_new.head(2)

,user_id,actual,K_15,K_20,K_25,K_30,K_35,K_40,K_45,K_50,K_55,K_60,K_65,K_70,K_75,K_80,K_85,K_90,K_95,K_100
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[999999, 1082185, 981760, 995242, 840361]","[999999, 1082185, 981760, 995242, 840361]","[999999, 1082185, 981760, 995242, 1127831]","[999999, 1082185, 981760, 995242, 1127831]","[999999, 1082185, 981760, 995242, 1127831]","[999999, 1082185, 981760, 995242, 1127831]","[999999, 1082185, 981760, 995242, 1127831]","[999999, 1082185, 981760, 995242, 1127831]","[999999, 1082185, 981760, 995242, 1127831]","[999999, 1082185, 981760, 995242, 1127831]","[999999, 1082185, 981760, 995242, 1127831]","[999999, 1082185, 981760, 995242, 1127831]","[999999, 1082185, 981760, 995242, 1127831]","[999999, 1082185, 981760, 995242, 1127831]","[999999, 1082185, 981760, 995242, 1127831]","[999999, 1082185, 981760, 995242, 1127831]","[999999, 1082185, 981760, 995242, 1127831]","[999999, 1082185, 981760, 995242, 1127831]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]"


#### Выбор оптимального числа K-соседей

In [979]:
# Оценка оптимального числа K-соседей
alg_list = result_new.columns[2:].tolist()

result_rating = []

for alg in alg_list:
    res = result_new.apply(lambda x: precision_at_k(x[alg], x['actual'],  5), axis = 1).mean()

    result_rating.append( [res, alg] )
    result_rating.sort(key = lambda x: x[0], reverse = True)
    
print(f'Место  Качество  K-соседей')
print()
for place, res in enumerate(result_rating):
    print(f'   {place + 1}  {round(res[0], 5)} \t{res[1]}')

Место  Качество  K-соседей

   1  0.15318 	K_15
   2  0.1526 	K_20
   3  0.15005 	K_25
   4  0.14946 	K_30
   5  0.14927 	K_35
   6  0.14907 	K_40
   7  0.14907 	K_45
   8  0.14907 	K_50
   9  0.14907 	K_55
   10  0.14907 	K_60
   11  0.14907 	K_65
   12  0.14907 	K_70
   13  0.14907 	K_75
   14  0.14907 	K_80
   15  0.14907 	K_85
   16  0.14907 	K_90
   17  0.14907 	K_95
   18  0.14907 	K_100


#### Предсказание ансамблем лучших алгоритмов

In [980]:
n_Best = 3
best_alg = [res[1] for res in result_test[:n_Best]]
print(f'Лучшие алгоритмы:\n\t{best_alg}')

Лучшие алгоритмы:
	['own_purchases', 'popular_recommendation', 'tfidf']


In [981]:
result[best_alg].head(3)

,own_purchases,popular_recommendation,tfidf
0,"[999999, 1082185, 1029743, 995785, 1004906]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 999999, 1098066]"
1,"[999999, 1082185, 1098066, 6534178, 1127831]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1098066, 826249, 999999]"
2,"[999999, 1082185, 1029743, 6534178, 1127831]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 999999, 878996]"


In [982]:
# итоговое предсказание по результатам лучших алгоритмов
def ensemble_top_alg(item_list):

    df = pd.DataFrame(item_list)
    top_items = df[0].value_counts().head(5)
    res = top_items.index.tolist()

    return res

In [983]:
# Предсказание ансамблем лучших алгоритмов
count = result.shape[0]
print('Кол-во строк =', count)

i = 0
for index, row in result.iterrows():
    i += 1
    res = str( ensemble_top_alg( sum(result.loc[index, best_alg].tolist(), [])))
    
    if (i % 100 == 0):
        print(f'\t{i}\t{res}')

    result.loc[index, 'ensemble_top_best_alg'] = res
        
# перевод строки с предсказаниями в список
column = 'ensemble_top_best_alg'    
result[column] = result[column].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))

Кол-во строк = 2042
	100	[999999, 1082185, 6534178, 1029743, 1098066]
	200	[1082185, 999999, 1127831, 1029743, 883404]
	300	[1082185, 999999, 6534178, 1029743, 1127831]
	400	[1082185, 999999, 1127831, 6534166, 962568]
	500	[1082185, 999999, 1029743, 6534178, 1098066]
	600	[1082185, 999999, 1127831, 6534178, 1098066]
	700	[1082185, 999999, 6534178, 1029743, 1127831]
	800	[1082185, 999999, 1098066, 6534178, 826249]
	900	[1082185, 999999, 1127831, 1029743, 1081177]
	1000	[1082185, 999999, 1127831, 1098066, 1068719]
	1100	[1082185, 999999, 6534178, 1029743, 1098066]
	1200	[1082185, 999999, 1027874, 826249, 1053690]
	1300	[1082185, 999999, 5569230, 1053690, 1127831]
	1400	[1082185, 999999, 1127831, 6534178, 6534166]
	1500	[1082185, 999999, 995785, 1127831, 1029743]
	1600	[1082185, 999999, 1098066, 1081177, 1053690]
	1700	[1082185, 999999, 6534178, 1098066, 826249]
	1800	[1082185, 999999, 6534178, 1029743, 1098066]
	1900	[1082185, 999999, 1127831, 1029743, 6534178]
	2000	[1082185, 999999, 65

In [984]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation,ensemble_top_best_alg
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1005456, 6942901, 10149303, 9424406, 904199]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[991546, 838797, 7441180, 838396, 975938]","[1082185, 999999, 1029743, 1127831, 995785]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[9803693, 1095915, 949877, 9884766, 6961764]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[976199, 1024231, 8090509, 972657, 908318]","[1082185, 999999, 6534178, 1098066, 1127831]"


In [985]:
alg_list = ['random_recommendation',
                'weighted_random_recommendation',
                'popular_recommendation',
                'itemitem',
                'cosine',
                'tfidf',
                'own_purchases',
                'ensemble_top_best_alg']

result_test_new = show_alg_result(alg_list)

Место  Качество  Алгоритм

   1   0.17969	 own_purchases
   2   0.17542	 ensemble_top_best_alg
   3   0.15524	 popular_recommendation
   4   0.13898	 tfidf
   5   0.13692	 itemitem
   6   0.13291	 cosine
   7   0.02165	 weighted_random_recommendation
   8   0.00127	 random_recommendation


#### Выводы
* Алгоритм, построенный на ансамбле из **пяти  лучших алгоритмов** показал не самый лучший результат - **4-й** (я не стал выводить эту таблицу)
* А алгоритм, построенный на ансамбле из **трех лучших алгоритмов** показал **2-й результат**, очень близкий к максимальному.

